In [9]:
import bokeh
import json
import pandas as pd
import numpy as np
import requests
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import NumeralTickFormatter
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure
from ARCTIC import hdf5_interface
from ARCTIC import nrel_api_interface

To call a number of different locations all at once, just call using a dictionary with the location names as the keys and have nested lat and lon keys therein, and then the lat and lon. Save the figures as the dictionary keys+ the .format construction that includes the tilt plot name or w/e. Also allow option to pass a list of tilts to make it customizable.

In [6]:
location_dataframe = pd.DataFrame(columns=['location','latitude','longitude'])
location_dataframe['location']=['Ambler-Shungnak-Kobuk','Anchorage','Bethel','Chickaloon',
                       'Deering','Denali Park','Fairbanks','Fort Yukon',
                       'Galena-Koyukuk-Ruby', 'Homer','Naknek','Noatak',
                       'Noorvik','Soldotna','Valdez','Wasilla-Palmer']

location_dataframe['latitude']=[66.995834, 61.193625, 60.794938, 61.823570,
                       66.069413, 63.537277, 64.838033, 66.571563,
                       64.782991, 59.652521, 58.728349, 67.570921,
                       66.836039, 60.486370, 61.128663, 61.582242]


location_dataframe['longitude']=[ -157.377096, -149.694974, -161.770716, -148.450442,
                        -162.766760,  -150.985453, -147.668970,  -145.250173,
                        -156.744933, -151.536496, -157.017444, -162.967490,
                         -161.041913, -151.060702, -146.353366, -149.441001]


location_dataframe

,location,latitude,longitude
0,Ambler-Shungnak-Kobuk,66.995834,-157.377096
1,Anchorage,61.193625,-149.694974
2,Bethel,60.794938,-161.770716
3,Chickaloon,61.823570,-148.450442
4,Deering,66.069413,-162.766760
5,Denali Park,63.537277,-150.985453
6,Fairbanks,64.838033,-147.668970
7,Fort Yukon,66.571563,-145.250173
8,Galena-Koyukuk-Ruby,64.782991,-156.744933
9,Homer,59.652521,-151.536496


In [13]:
#This will be the tilt angle framework

#def tilt_angle_plot_generation(location_dataframe, tilt_list, weather_dataset):

#Pass the list of tilts you want to use. 
# We choose the tilt at the below degree, and the Fairbanks lon & lat are (64.82,-147.87)
tilt_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]

weather_dataset = 'tmy2'
#For loop to walk through each row in the location dataframe, calling the below with it.
#for i in range(len(locations.index)):

# Call the PV Watts to get the data (prediction)
nrel_long_tilt = []

for i in range(len(tilt_list)):
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilt_list[i], "azimuth": 180, "lat": location_dataframe['latitude'][1], "lon": location_dataframe['longitude'][1], "dataset": weather_dataset}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_long_tilt.append(new_dataframe)

tilt_response_dataframe = pd.DataFrame(columns = tilt_list)
for i, tilt in enumerate(tilt_list):
    tilt_response_dataframe[tilt] = nrel_long_tilt[i]['ac_monthly']
tilt_response_dataframe

,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90
0,36.584713,43.631260,50.557304,57.143124,63.274998,68.892509,73.967133,78.477638,82.413078,85.765381,88.527298,90.693787,92.261330,93.227859,93.593102,93.359154,92.530769,91.113022
1,106.631058,120.979492,134.536209,147.085785,158.513626,168.758804,177.789246,185.587540,192.142944,197.449127,201.502579,204.302841,205.853119,206.156677,205.221252,203.055405,199.671005,195.084976
2,258.263519,280.689148,300.999939,319.112610,334.986359,348.585541,359.903137,368.930542,375.662750,380.000946,382.023560,381.829346,379.454010,374.912811,368.172577,359.123627,347.872742,334.479126
3,374.560394,391.574463,406.106384,418.150543,427.725250,434.844635,439.396545,441.421783,440.998138,438.154541,432.999023,425.532043,415.730835,403.634308,389.342987,372.773926,354.078827,333.357452
4,505.933380,519.785950,530.752136,538.883545,544.013123,546.136230,545.336426,541.757446,535.472778,526.374695,514.362183,499.513550,481.927856,461.732819,439.083679,414.260223,387.364075,358.424164
5,507.983795,515.729065,520.874390,523.567993,523.928894,521.606445,516.665894,509.245148,499.622192,487.822174,473.551361,456.853821,437.862854,416.751801,393.766205,369.260620,343.265228,315.518433
6,482.193817,491.016174,497.319611,501.249969,502.741608,501.627167,497.981567,491.971100,483.666962,473.140259,460.192657,444.863220,427.264954,407.542511,385.978668,362.614044,337.676544,311.079590
7,359.503143,371.415314,381.204437,388.829315,394.314850,397.679901,398.985138,398.240509,395.406158,390.505188,383.552521,374.594666,363.683746,350.886627,336.276093,319.951599,301.988739,282.492401
8,243.958893,260.429840,275.080933,287.876251,298.800262,307.832062,314.960938,320.187500,323.512726,324.943085,324.489258,322.168304,317.995392,311.995300,304.194092,294.624084,283.317719,270.315094
9,116.928299,129.387802,141.000488,151.623947,161.180145,169.626099,176.935547,183.094925,188.094482,191.927963,194.593719,196.093323,196.431732,195.616562,193.657867,190.567703,186.361176,181.059830
